In [1]:
import requests
import json
import pandas as pd
import time

# Running flask app from powershell
# $env:FLASK_APP = "api"
# $env:FLASK_ENV = "development"
# flask run

# Running flask from terminal on OSX
# export FLASK_APP=api
# export FLASK_ENV=development
# flask run

# Running with gunicorn (heroku, digitalocean, etc):
# Procfile >
# web: gunicorn wsgi --log-file -
# wsgi.py >
# from api import app as application
#
#if __name__ == "__main__":
#    application.run()

# https://devcenter.heroku.com/articles/getting-started-with-python
# https://www.digitalocean.com/community/tutorials/how-to-serve-flask-applications-with-gunicorn-and-nginx-on-ubuntu-14-04

In [2]:
# Set up heroku database > 
# heroku addons:create heroku-postgresql:hobby-dev
# Postgres db now up
# Look at config vars (should have DATABASE_URL) >
# heroku config
# Connect to remote psql > 
# heroku pg:psql
#


In [4]:
#  baseurl = 'https://vast-depths-89393.herokuapp.com/'
baseurl = 'http://127.0.0.1:5000/'

In [5]:
url = baseurl +'createUser'
headers = {'content-type': 'application/json'}
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)

In [6]:
response.json()

{'signingKey_hex': 'c880fc9e70e9e7e183ab29228ea7a6b0350f3587a161e02e287209b2b233ae02',
 'traderId': '4',
 'verifyKey_hex': '9733e0978de43fd1a98b0a5ca48a3e8c460638a14545dfcb844d8f6e160e0690'}

In [7]:
url = baseurl + 'createMarket'

mkId = 6
# copy from response
sk = response.json()['signingKey_hex']
vk = response.json()['verifyKey_hex']
tId = response.json()['traderId']

In [10]:
content_makemarket = {"signingKey_hex": sk,
                    "traderId": tId, 
                    "verifyKey_hex": vk,
                    "marketRootId": mkId, 
                    "marketBranchId": 1, 
                    "marketMin": 0,
                    "marketMax":1}

response = requests.post(url, data=json.dumps(content_makemarket), headers=headers)

In [11]:
pd.DataFrame(response.json(), index=[0])


,checks,marketBranchId,marketMax,marketMin,marketRootId,traderId
0,ProbablyASignatureError,1,1,0,6,5


In [17]:
# Create a trade
url = baseurl+'createTrade'

content_maketrade = {"signingKey_hex": sk, "traderId": tId, "verifyKey_hex": vk,
  "marketRootId": mkId, "marketBranchId": 1, "price": 0.5678, "quantity":1}

In [18]:
# Post trade
response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)

In [19]:
# View json response
pd.DataFrame(response.json(), index=[0])

,checks,marketBranchId,marketRootId,price,quantity,traderId
0,"(True, True)",1,6,0.5678,1,4


In [20]:
# Create a matched trade
url = baseurl+'createTrade'
content_maketrade['quantity'] = -1
response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)

In [21]:
response.json()

{'checks': 'ProbablyASignatureError',
 'marketBranchId': 1,
 'marketRootId': 6,
 'price': 0.5678,
 'quantity': -1,
 'traderId': '4'}

In [22]:
# Create many matched trades
url = baseurl+'createTrade'

for i in range(10):
    t = time.time()
    content_maketrade['quantity'] = -1
    response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)
    content_maketrade['quantity']=1
    response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)
    print('matched trade: ' + str(i) + ' in ' + str(time.time() - t) + ' seconds.')


matched trade: 0 in 7.776038885116577 seconds.
matched trade: 1 in 7.740344047546387 seconds.
matched trade: 2 in 7.798087120056152 seconds.
matched trade: 3 in 7.806659698486328 seconds.
matched trade: 4 in 8.25097393989563 seconds.
matched trade: 5 in 8.011950731277466 seconds.
matched trade: 6 in 7.78582763671875 seconds.
matched trade: 7 in 8.261141061782837 seconds.
matched trade: 8 in 8.275362014770508 seconds.
matched trade: 9 in 7.916067838668823 seconds.


In [23]:
url = baseurl+'createMarket'

for i in range(4):
    t = time.time()

    content_makemarket = {"signingKey_hex": sk,
      "traderId": int(tId), "verifyKey_hex": vk,
      "marketRootId": i+1, "marketBranchId": 1, "marketMin": 0, "marketMax":0.7}

    response = requests.post(url, data=json.dumps(content_makemarket), headers=headers)
    if response.json()['checks'] == 'True':
        print('market created: ' + str(i+1) + ' in ' + str(time.time() - t) + ' seconds.')
    else:
        print('failed to create market ' +str(i+1))
    

failed to create market 1
failed to create market 2
failed to create market 3
failed to create market 4


In [166]:
response.json()

{'checks': 'ProbablyASignatureError',
 'marketBranchId': 1,
 'marketRootId': 6,
 'price': 0.5678,
 'quantity': 1,
 'traderId': '39'}

In [24]:
url = baseurl+'viewMarketBounds'
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)



In [25]:
a = pd.read_json(response.json())
a

,marketRootId,marketBranchId,marketMin,marketMax
0,6,1,0.0,1.0
1,1,1,0.0,1.0
2,1,2,0.1,0.9
3,2,1,0.0,1.0
4,3,1,0.0,1.0
5,4,1,0.0,1.0
6,5,1,0.0,1.0


In [26]:
url = baseurl+'viewOrderBook'
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)

In [27]:
a = pd.read_json(response.json())
a

,marketRootId,marketBranchId,price,quantity,traderId
0,6,1,0.5678,1,4
1,6,1,0.5678,-1,4
10,6,1,0.5678,1,4
11,6,1,0.5678,-1,4
12,6,1,0.5678,-1,4
13,6,1,0.5678,1,4
14,6,1,0.5678,-1,4
15,6,1,0.5678,1,4
16,6,1,0.5678,1,4
17,6,1,0.5678,-1,4


In [170]:
# Create many matched trades
url = baseurl+'createTrade'

for i in range(10):
    t = time.time()
    content_maketrade['quantity'] = 1
    content_maketrade['price'] = 0.44
    response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)
    print('matched trade: ' + str(i) + ' in ' + str(time.time() - t) + ' seconds.')


matched trade: 0 in 24.23835015296936 seconds.
matched trade: 1 in 18.970323085784912 seconds.
matched trade: 2 in 19.54018783569336 seconds.
matched trade: 3 in 20.687762022018433 seconds.
matched trade: 4 in 18.369783639907837 seconds.
matched trade: 5 in 18.592928886413574 seconds.
matched trade: 6 in 29.398239850997925 seconds.
matched trade: 7 in 31.328058004379272 seconds.
matched trade: 8 in 30.595832109451294 seconds.
matched trade: 9 in 19.711176872253418 seconds.


In [28]:
url = baseurl+'viewOpenTrades'

content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)

In [29]:
pd.read_json(response.json())

,marketRootId,marketBranchId,price,quantity,traderId


In [30]:
url = baseurl+'viewMatchedTrades'
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)

In [31]:
pd.read_json(response.json())

,marketRootId,marketBranchId,price,quantity,traderId
0,6,1,0.5678,0,4


In [99]:
tId

'6'